In [1]:
import time
import requests
from pprint import pprint
import pandas as pd
import numpy as np
import os
import datetime

In [2]:
# setting chrome driver according to this thread: https://stackoverflow.com/questions/29858752/error-message-chromedriver-executable-needs-to-be-available-in-the-path

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


In [3]:
# defining competition and user class

class kaggle_competition:
    link = ''
    title = ''
    description = ''
    prize = 0
    holder = ''
    team_count = 0
    tags = []
    category  = 0
    teams = []
    completed = 0
    deadline = ''
    def __init__(self,link):
        self.link = link
        self.title = ''
        self.description = ''
        self.prize = 0
        self.holder = ''
        self.team_count = 0
        self.tags = []
        self.category  = 0
        self.teams = []
        self.completed = 0
        self.deadline = ''
    def to_dict(self):     
        return {
            'link': self.link,
            'title': self.title,
            'description': self.description,
            'prize': self.prize,
            'team_count': self.team_count,
            'tags': self.tags,
            'category': self.category,
            'completed': self.completed,
            'deadline': self.deadline,
            'holder': self.holder
        }       
    
    
class kaggle_user:
    link = ''
    name = ''
    company = ''
    location = ''
    time_joined = 0
    linkedin_account = ''
    twitter_account = ''
    github_account = ''
    progression = ''
    competitions_rank = 0
    competitions_gold_medal = 0
    competitions_silver_medal = 0
    competitions_bronze_medal = 0
    bio = ''
    competitions = []
    def __init__(self,link):
        self.link = link
    def to_dict(self):     
        return {
            'link': self.link,
            'name': self.name,
            'company': self.company,
            'location': self.location,
            'time_joined': self.time_joined,
            'linkedin_account': self.linkedin_account,
            'twitter_account': self.twitter_account,
            'github_account': self.github_account,
            'progression': self.progression,
            'competitions_rank': self.competitions_rank,
            'competitions_gold_medal': self.competitions_gold_medal,
            'competitions_silver_medal': self.competitions_silver_medal,
            'competitions_bronze_medal': self.competitions_bronze_medal,
            'competitions': self.competitions,
            'bio': self.bio
        }       


In [4]:
driver = webdriver.Chrome("C:/Users/Ramin.Forouzandeh17/.wdm/drivers/chromedriver/77.0.3865.40/win32/chromedriver.exe") #opens a chrome instance
driver.get("https://www.kaggle.com/competitions") #opens website

#####################################scroll to the bottom of the infinite page
SCROLL_PAUSE_TIME = 0.5
last_height = driver.execute_script("return document.body.scrollHeight")    # Get scroll height
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    # Scroll down to bottom
    time.sleep(SCROLL_PAUSE_TIME)    # Wait to load page
    new_height = driver.execute_script("return document.body.scrollHeight")    # Calculate new scroll height and compare with last scroll height
    if new_height == last_height:
        break
    last_height = new_height

##################################### get competition links and store them in a list
competitions_link_list = []
competition_link = driver.find_elements_by_class_name('CompetitionListItem_CompetitionTitle-sc-1gtde6t.fgUUdY') #all competitions
for i in range(0,len(competition_link)):
    competitions_link_list.append(competition_link[i].get_attribute("href"))


In [5]:
##################################### get competitions information  
competitions = []
competitions_dict = []
  
for i in range(0,len(competitions_link_list)):
    try:
        competition = kaggle_competition(competitions_link_list[i])
        driver.get(competition.link) #opens website
        time.sleep(1)
        if len(driver.find_elements_by_class_name('competition-header__title'))>0:
            competition.title = driver.find_element_by_class_name('competition-header__title').text
        if len(driver.find_elements_by_class_name('competition-header__subtitle'))>0:
            competition.description = driver.find_element_by_class_name('competition-header__subtitle').text
        if len(driver.find_elements_by_class_name('competition-header__prize-value'))>0:
            competition.prize = int(''.join(i for i in driver.find_element_by_class_name('competition-header__prize-value').text if i.isdigit()))
        if len(driver.find_elements_by_class_name('horizontal-list-item.horizontal-list-item--bullet.horizontal-list-item--default'))>0:
            competition.holder = driver.find_elements_by_class_name('horizontal-list-item.horizontal-list-item--bullet.horizontal-list-item--default')[0].text
        if len(driver.find_elements_by_class_name('competition-overview__stats'))>0:
            competition.team_count = int(''.join(i for i in driver.find_element_by_class_name('competition-overview__stats-section').find_element_by_class_name('competition-overview__stat-number').text if i.isdigit()))
            try:
                deadline_str = driver.find_elements_by_class_name('horizontal-list-item.horizontal-list-item--bullet.horizontal-list-item--default')[2].find_element_by_tag_name('span').find_element_by_tag_name('span').get_attribute("title").split(' (', 1)[0]   
                competition.deadline = datetime.datetime.strptime(deadline_str, '%a %b %d %Y %H:%M:%S %Z%z')
                if competition.deadline < datetime.datetime.now(datetime.timezone.utc):
                    competition.completed = 1
                else:
                    competition.completed = 0
            except:    # if format of date is different than ususal
                pass

        for elm in driver.find_elements_by_class_name('CategoryButton_CategoryName-sc-c10946.eLztBP'):
            competition.tags.append(elm.text)
        if len(driver.find_elements_by_class_name('competition-header__classification-text'))>0:
            competition.category = driver.find_element_by_class_name('competition-header__classification-text').text

        #### teams and users in the competition

        driver.get(competition.link+'\leaderboard') #opens leaderboard page 
        time.sleep(1)
        if len(driver.find_elements_by_class_name('competition-leaderboard__load-more-count'))>0:
            driver.find_element_by_class_name('competition-leaderboard__load-more-count').click()  # click on "load more" link at the bottom
        time.sleep(5)    # wait for the page to load
        leaderboard_rows = driver.find_elements_by_class_name('competition-leaderboard__row')

        competition.teams = []
        for row in range(0,len(leaderboard_rows)):
            team = {}
            team["rank"] = int(leaderboard_rows[row].find_element_by_class_name('competition-leaderboard__td-rank').text)
            team["name"] = leaderboard_rows[row].find_elements_by_tag_name('td')[1].text
            team["score"] = leaderboard_rows[row].find_element_by_class_name('competition-leaderboard__td-score').text
            team["last_entry"] = leaderboard_rows[row].find_element_by_class_name('competition-leaderboard__td-last').text
            team["members"] = []
            for elm in leaderboard_rows[row].find_elements_by_class_name("avatar"):
                team["members"].append(kaggle_user(elm.get_attribute("href")))   # team members stored as kaggle_user objects with their link as input
            competition.teams.append(team)

        #save competition in list
        competitions.append(competition)
        competitions_dict.append(competition.to_dict()) 
    except (IndexError):
        print(competition.link)
        continue
        
####### change the list of objects into dataframe    
competitions_df = pd.DataFrame(competitions_dict)
competitions_df.T.to_csv(r'Kaggle_competition_list.csv',index=False, header=False)


In [6]:
all_competitions = competitions

In [7]:
##################################### get users' information for each competition  
Kaggle_users = []

for i in range(0,len(competitions)):
    competition = competitions[i]
    for j in range(0,len(competition.teams)):
        team = competition.teams[j]
        for k in range(0,len(team['members'])):
            team_member = team['members'][k]         #this is a variable in kaggle_user class
            
            #store in a dictionary
            Kaggle_users.append(team_member.to_dict())
            
####### change the list of objects into dataframe    
Kaggle_users = pd.DataFrame(Kaggle_users)
Kaggle_users.T.to_csv(r'Kaggle_users.csv',index=False, header=False)


In [ ]:
dir(team_member)

In [ ]:
dir(competition)

In [ ]:
competition.link